In [1]:
import tensorflow as tf
import gym
import numpy as np
from tensorflow_probability import distributions as dists
import tensorflow.keras.layers as kl

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

tf.random.set_seed(0)

In [2]:
env = gym.make('Pendulum-v0')
is_continuous = isinstance(env.action_space, gym.spaces.Box)

In [3]:
action_dim = env.action_space.shape[0]
print(action_dim)

1


In [4]:
from rl_agents.ppo.policy import ContinuousSample

In [8]:
a = env.action_space.sample()

dist_layer = ContinuousSample(action_dim)

y = dist_layer(np.array([a, a]))

print(y[0].numpy())
print(y[1].numpy())
print(y[2])
print(y[3].numpy())

[[2.6097894]
 [2.0963311]]
[[-0.95805526]
 [-0.4078633 ]]
tfp.distributions.Normal("ContinuousSample/Normal/", batch_shape=[2, 1], event_shape=[], dtype=float32)
[[0.588605]]


In [6]:
np.exp(-0.53)

0.5886049696783552

In [9]:
from rl_agents.ppo.policy import Actor

In [13]:
env.observation_space

Box(3,)

In [25]:
actor = Actor(env.observation_space.shape, env.action_space.shape, is_continuous)

obs = env.reset()
sample, log_prob, dist, std, logits = actor(obs[None])
print(sample, log_prob, dist, std, logits)

tf.Tensor([[0.20062875]], shape=(1, 1), dtype=float64) tf.Tensor([[-0.4194354]], shape=(1, 1), dtype=float64) tfp.distributions.Normal("Actor/ContinuousSample/Normal/", batch_shape=[1, 1], event_shape=[], dtype=float64) <tf.Variable 'Variable:0' shape=(1, 1) dtype=float64, numpy=array([[0.58860497]])> tf.Tensor([[0.05526152]], shape=(1, 1), dtype=float64)
